In [1]:
from urllib.request import urlopen,urlretrieve
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
#폴더생성
import os
import pandas as pd

In [2]:
#포스터 이미지 저장
def image_save(img_url, i):
    img_folder_path='c:/chj/images/naver_movie' #이미지 저장 폴더

    if not os.path.isdir(img_folder_path): #없으면 새로 생성
        os.mkdir(img_folder_path)
    savename="c:/chj/images/naver_movie/movie"+str(i)+'.jpg'
    urlretrieve(img_url,savename)
        

In [3]:
def main():
    URL='https://movie.naver.com/movie/running/current.nhn'
    #selenium(동적 스크래핑)
    driver = webdriver.Chrome('chromedriver')
    driver.implicitly_wait(5)
    driver.get(URL)
    time.sleep(2) 
    
    summary_list=[]
    img_list=[]

    for i in range(1, 110):
       
        clickLink = driver.find_element_by_css_selector('#content > div.article > div:nth-child(1) > div.lst_wrap > ul > li:nth-child('+str(i)+') > div > a')
        clickLink.click()
        summary = driver.find_element_by_css_selector('#content > div.article > div.section_group.section_group_frst > div:nth-child(1) > div > div')
        summary_list.append(summary.text.replace("\n","").replace("줄거리","").replace("\'",""))
        img= driver.find_element_by_css_selector('#content  div.poster > a > img')
        img_url = img.get_attribute('src')
        image_save(img_url,i)
        img_list.append(img_url)
        time.sleep(1)
        driver.back()
        time.sleep(1)
        
		
    #beautifulSoup(정적 스크래핑)
    html=urlopen(URL)
    bs=BeautifulSoup(html, 'html.parser')
    movie_list=bs.select('div.lst_wrap>ul.lst_detail_t1>li')
    tit_list=[]
    title_list=[]
    point_list=[]
    genre_list=[]
    time_list=[]
    playday_list=[]
    director_list=[]
    actors_list=[]
    actors_small_list=[]
    
    i=0
    for movie in movie_list:
        img_url=movie.img['src']
        title_list.append(movie.select('dl > dt > a')[0].string)
        tit_list.append(movie.find('span').string)
        point_list.append(movie.select('dl > dd.star > dl.info_star > dd > div > a > span.num')[0].text) 
        genre_list.append(movie.select('dl > dd:nth-child(3) > dl > dd > span.link_txt>a')[0].string)
        strings=movie.select('dl > dd:nth-child(3) > dl >dd:nth-child(2)')[0].text.replace("\t","").replace("\r","").replace("\n","")
        time_list.append(strings[-18:-14].replace("|",""))
        playday_list.append(strings[-13:])
        director_list.append(movie.select('dd:nth-child(3) > dl > dd:nth-child(4) > span>a')[0].string)
        actors=movie.select('dd:nth-child(3) > dl >dd:nth-child(6)>span>a')
        atr=[]
        for actor in actors:
            atr.append(actor.text)
        actors_list.append(atr)
       
        i+=1;
    
    
    data={'영화제목':title_list, '영화 관람등급':tit_list,'평점':point_list,'장르':genre_list,'러닝타임':time_list,'개봉일':playday_list,'감독':director_list,'출연':actors_list, '줄거리':summary_list, '포스터 이미지 주소':img_list}
    df=pd.DataFrame(data)
    print(df)
    df.to_csv('c:/chj/data/naver_moive.csv', index=False)

#     print(title_list)
#     print(tit_list)
#     print(point_list)
#     print(genre_list)
#     print(time_list)
#     print(playday_list)
#     print(director_list)
#     print(actors_list)
#     print(summary_list)
#     print(img_list)

In [4]:
main()

                       영화제목  영화 관람등급    평점     장르  러닝타임            개봉일  \
0                       미나리  12세 관람가  8.20    드라마  115분  2021.03.03 개봉   
1               라야와 마지막 드래곤   전체 관람가  8.97  애니메이션  114분  2021.03.04 개봉   
2         극장판 귀멸의 칼날: 무한열차편  15세 관람가  9.25  애니메이션  117분  2021.01.27 개봉   
3                        소울   전체 관람가  9.31  애니메이션  107분  2021.01.20 개봉   
4                      중경삼림  15세 관람가  8.91    드라마  102분  2021.03.04 개봉   
..                      ...      ...   ...    ...   ...            ...   
104                 구라, 베토벤  15세 관람가  8.92     봉수  101분  2021.02.25 개봉   
105  디지몬 어드벤처 라스트 에볼루션 : 인연  12세 관람가  9.55  애니메이션  114분  2021.02.17 개봉   
106      미스 사이공: 25주년 특별 공연  15세 관람가  9.23   공연실황  175분  2016.11.24 개봉   
107                    일대종사  12세 관람가  8.01     무협  122분  2013.08.22 개봉   
108            잃어버린 얼굴 1895   전체 관람가  7.35    뮤지컬  148분  2021.02.24 개봉   

           감독                                                 출연  \
0         정이삭                [스티븐 연, 한예리, 윤